In [ ]:
#default_exp status_filter

# Status Filter

> Filter a bundle of medications by status.

In [ ]:
#export
from vulcan_medication_bundle.core import *

In [ ]:
import json

### Medication status filter

Remove medication if the status tells us it was not or will not be taken. 

- https://www.hl7.org/fhir/valueset-medicationrequest-status.html
- https://www.hl7.org/fhir/valueset-medicationdispense-status.html
- https://www.hl7.org/fhir/valueset-medication-admin-status.html
- https://www.hl7.org/fhir/valueset-medication-statement-status.html


#### Statuses that we want to remove from the bundle

# TODO: update using med-status.xlsx values

- MedicationRequest (Include: active, on-hold, completed, entered-in-error, unknown)
    - cancelled 
        - The prescription has been withdrawn before any administrations have occurred
    - stopped 
        - Actions implied by the prescription are to be permanently halted, before all of the administrations occurred. 
        - TODO: This is a ? **halted, before all ...** i.e. might some of the administrations occured
    - draft
        - The prescription is not yet 'actionable'
     
     
- MedicationDispense (Include: on-hold, completed, unknown)
    - preparation	
        - The core event has not started yet, 
    - in-progress
        - The dispensed product is ready for pickup
    - cancelled	
        - The dispensed product was not and will never be picked up by the patient
    - entered-in-error	
        - The dispense was entered in error and therefore nullified
    - stopped	
        - Actions implied by the dispense have been permanently halted, before all of them occurred
        - TODO: This is a ? **hatled, before all ...** i.e. might some of the actions occured
    - declined	
        - The dispense was declined and not performed.
        
- MedicationAdministration (Include: in-progress, on-hold, completed, stopped, unknown)
    - not-done	
        - The administration was terminated prior to any impact on the subject
    - entered-in-error	
        - The administration was entered in error and therefore nullified
        
- MedicationStatement (Include: active, completed, entered-in-error, intended, stopped, on-hold, unknown)
    - not-taken
        - The medication was not consumed by the patient

#### What if these statuses are not appropriate for every study?

It's possible that a study needs to see medication records confirming that a medication was not taken.

i.e. If previous treatment with a medication is an exclusion criteria, absense of a medication record might not be enough to be sure the patient didn't take it.

So we'll need to make filters configurable ...

#### Should we always run the status filter?

The CMOCCUR part of FHIR-to-CDISC Mappings xlsx includes status filtering instructions.

i.e We might not want to implement status filtering on the patient medication bundle.

So we'll need to make filters optional ...

In [ ]:
#export
# Note: this is using med-status.xlsx values
CM_EXCLUDE_STATUS_MAP = dict(
        MedicationRequest=['cancelled','entered-in-error','stopped','draft'],
        MedicationDispense=['cancelled','entered-in-error','stopped','declined'],
        MedicationAdministration=['not-done','entered-in-error'],
        MedicationStatement=['entered-in-error','not-taken'])

In [ ]:
#export
def _new_list_entry(reference, resourceType, status, statuses_to_remove):
    return {'item': {'reference': reference},
            'flag': {'text': f"{resourceType} negated as its status '{status}' is one of {statuses_to_remove}"}}

def get_negated_list(bundle, exclude_status_map=CM_EXCLUDE_STATUS_MAP):
    result = new_list('Negated list of medications')
    # put the status map in an annotation
    result['note'] = [dict(text=f'Exclude status map: ```{exclude_status_map}```')]
    for entry in bundle.get('entry', []):
        resource = entry.get('resource', {})
        resourceType, status = resource.get('resourceType'), resource.get('status')
        statuses_to_remove = exclude_status_map.get(resourceType)
        if statuses_to_remove is not None and status in statuses_to_remove:
            result['entry'].append(_new_list_entry(
                entry['fullUrl'], resourceType, status, statuses_to_remove))
    if not result['entry']: del result['entry']
    return result

In [ ]:
print(json.dumps(get_negated_list({}), indent=2)) # https://inferno.healthit.gov/validator/results

{
  "resourceType": "List",
  "id": "b6c19ca6-3da4-423e-b8fe-6bf050b19b14",
  "title": "Negated list of medications",
  "status": "current",
  "mode": "snapshot",
  "date": "2021-09-15T10:54:01Z",
  "note": [
    {
      "text": "Exclude status map: ```{'MedicationRequest': ['cancelled', 'entered-in-error', 'stopped', 'draft'], 'MedicationDispense': ['cancelled', 'entered-in-error', 'stopped', 'declined'], 'MedicationAdministration': ['not-done', 'entered-in-error'], 'MedicationStatement': ['entered-in-error', 'not-taken']}```"
    }
  ]
}


In [ ]:
with open('test/patient_medication_bundle_2.json') as f:
    bundle = json.load(f)

In [ ]:
print(json.dumps(get_negated_list(bundle), indent=2)) # https://inferno.healthit.gov/validator/results

{
  "resourceType": "List",
  "id": "d92d687f-d262-4817-8d59-731ad62e8c98",
  "title": "Negated list of medications",
  "status": "current",
  "mode": "snapshot",
  "date": "2021-09-15T10:54:01Z",
  "entry": [
    {
      "item": {
        "reference": "https://server.fire.ly/r4/MedicationRequest/07f4d7c2-edef-45f2-b4cf-36bb2bf939cb"
      },
      "flag": {
        "text": "MedicationRequest negated as its status 'stopped' is one of ['cancelled', 'entered-in-error', 'stopped', 'draft']"
      }
    },
    {
      "item": {
        "reference": "https://server.fire.ly/r4/MedicationRequest/0c7eb3f2-96a6-4145-9c48-717adc11fa6b"
      },
      "flag": {
        "text": "MedicationRequest negated as its status 'draft' is one of ['cancelled', 'entered-in-error', 'stopped', 'draft']"
      }
    },
    {
      "item": {
        "reference": "https://server.fire.ly/r4/MedicationRequest/5f2c9f0e-0e27-4327-9367-a349269e5e86"
      },
      "flag": {
        "text": "MedicationRequest negated as

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_per_patient.ipynb.
Converted 20a_status_filter.ipynb.
Converted 30_cli.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_demo.ipynb.
Converted index.ipynb.
